In [ ]:
## import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
# from google.colab import files  ## to export our files to our computer drive
import random  ##iloc/loc

## Let's import the required libaries to create a delay
from random import randrange ##  allows us to randomize numbers library
import time ## time tracker

## Single page scrape

We want to scrape a table that contains NFL player salaries for 2019. 

The webpage is ```https://sandeepmj.github.io/scrape-example-page/"```

In [ ]:
##scrape url website
url = "https://sandeepmj.github.io/scrape-example-page"
page = requests.get(url)
print(page.status_code)  ## should print 200. checks http response code status


In [ ]:
## turn into soup
soup = BeautifulSoup(page.content, "html.parser")
print(type(soup))

In [ ]:
## MUST turn html into a string
html = soup.prettify()
print(type(html))

In [ ]:
## use Pandas to read tables on page
df = pd.read_html(html)
df

In [ ]:
## Do we want the first table?
df[0]

In [ ]:
## Do we want the the second table?
df[1]

In [ ]:
## store it into a copy called nfl_df
fl_df = df[1].copy() 
fl_df

## Multiple page scrape

We want to scrape the tables scattered across this site

https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html



In [2]:
## How is it different?
url = "https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page{}.html"



### We know we need a placeholder value of upto ```4```

### Let's create a variable called  ```total_pages``` to match number of pages on site.


In [3]:
## total pages to scrape
total_pages = 5

In [4]:
## Let's write the for loop
## but instead of storing into a list, we just feed it directly to our placeholder
## we want to just scape each page
for url_number in range(0,total_pages):
    link = url.format(url_number)
    print(link)
    site = requests.get(link)
    print(site.status_code)

https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page0.html
404
https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html
200
https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page2.html
200
https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page3.html
200
https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page4.html
200


In [6]:
total_pages = 5 ## number of pages we want to scrape
df_all = []
for url_number in range(1,total_pages):
    link = url.format(url_number)
    print(url_number)
#     print(f"I'm on page {page}")
    site = requests.get(link)
    try: 
        if site.status_code == 200:
            print(f"got it...scraping page...{link}")
            soup = BeautifulSoup(site.content, "html.parser")
            ## turn soup into html 
            html = soup.prettify()
            print(type(html))
            df = pd.read_html(html)
            df_all.append(df[0])

            
            
            snooze = randrange(5,7)
            print(f"snoozing for {snooze} seconds before scraping next link.")
            time.sleep(snooze)
            counter+=1

        else:
            print(f"oh no! {link} returned:", site.status_code)
    except: 
        print(f"I can't seem to find these urls")
        
df_all

1
got it...scraping page...https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page1.html
<class 'str'>
snoozing for 5 seconds before scraping next link.
I can't seem to find these urls
2
got it...scraping page...https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page2.html
<class 'str'>
snoozing for 5 seconds before scraping next link.
I can't seem to find these urls
3
got it...scraping page...https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page3.html
<class 'str'>
snoozing for 5 seconds before scraping next link.
I can't seem to find these urls
4
got it...scraping page...https://sandeepmj.github.io/scrape-example-page/heaviest-animals-page4.html
<class 'str'>
snoozing for 6 seconds before scraping next link.
I can't seem to find these urls


[                 Animal  Weight in KGs    Type
 0            Blue whale         136000  Marine
 1         Bowhead whale         100000  Marine
 2             Fin whale          70000  Marine
 3  Southern right whale          45000  Marine
 4        Humpback whale          30000  Marine,
                  Animal  Weight in kilograms    Type
 0            Gray whale                28500  Marine
 1  Northern right whale                23000  Marine
 2             Sei whale                20000  Marine
 3         Bryde's whale                16000  Marine
 4  Baird's beaked whale                11380  Marine,
                       Animal  Weight (KG)         Type
 0                Minke whale         7500       Marine
 1  Northern bottlenose whale         6500       Marine
 2     Gervais's beaked whale         5600       Marine
 3           African elephant         4800  Terrestrial
 4               Killer whale         3988       Marine,
                      Animal  Weight in kilograms

In [11]:
## write function to combine tabula tables into a single csv
## write function to combine tabula tables into a single csv
def combine_tables(list_name,filename):
  '''
  Takes tables produced by tabula and combines into a single CSV.
  Arguments: name of list produced by tabula and the CSV name you want (in quotes as a string)
  '''
  dataframes = [pd.DataFrame(a_table) for a_table in list_name] ## list comprehension to turn each tabula table into a dataframe
  df = pd.concat(dataframes) ## join/concat all the dataframes into one dataframe
  df.reset_index(inplace = True, drop = True) ## reset index, drop what was there before
  df.to_csv(filename, encoding='utf-8', index=False) ## convert that single dataframe into a csv
#   files.download(filename) ## download it
  print(f"{filename} is in your downloads folder!")

In [12]:
combine_tables(df_all,"animals_today.csv")

animals_today.csv is in your downloads folder!
